# MSSDS 434 FX_RL Learning Project

### Project overview
#### The application will follow this trajectory:
- User uses front end (VueJS or React) to select trading pair and set certain model criteria.
- Front end tells flask backend what the model parameters will be based on user selection.
- Flask backend triggers a callout to a Financial Data API (Quandl, IEX, IBKR, Yahoo, Google)
- Data is loaded into your classes to train a model based on the user inputs in step 1.
- The model trains on each hour up to the penultimate hour (h - 1), when it is then tested on on the last hour of data. 
- Results are fed back into Flask and through to the front end to display results.
- The user is given the option of productionalizing the model with the connected trading account using the current incoming data from whichever real-time financial data api has been chosen.
- The results of the productionalized model are published in real time.

#### Things that must be configurable via REST API
- Currency Pair
- Which side of the currency pair (right,left)
- Amount of hist data to train on: (1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max)
   - Window Size must infer this same value
- Price prediction in time interval window
- Return buy or sell command



### Imports

In [63]:
import gym
import gym_anytrading
import yfinance as yf
import pandas as pd
import os
import datetime
import matplotlib.pyplot as plt

In [64]:
#set alphavantage api-key
os.environ['ALPHAVANTAGE_API_KEY'] = '95O0QW8Y4UM8U9WA'
os.environ['WINDOW_SIZE'] = "15"
os.environ['HISTORICAL_PERIOD'] = "7d"
os.environ['HISTORICAL_INTERVAL'] = "1m"

In [65]:
curr_pairs = {
"EUR/USD":"EURUSD=X",
"USD/JPY":"JPY=X",
"GBP/USD":"GBPUSD=X",
"AUD/USD":"AUDUSD=X",
"NZD/USD":"NZDUSD=X",
"EUR/JPY":"EURJPY=X",
"GBP/JPY":"GBPJPY=X",
"EUR/GBP":"EURGBP=X",
"EUR/CAD":"EURCAD=X",
"EUR/SEK":"EURSEK=X",
"EUR/CHF":"EURCHF=X",
"EUR/HUF":"EURHUF=X",
"EUR/JPY":"EURJPY=X",
"USD/CNY":"CNY=X",
"USD/HKD":"HKD=X",
"USD/SGD":"SGD=X",
"USD/INR":"INR=X",
"USD/MXN":"MXN=X",
"USD/PHP":"PHP=X",
"USD/IDR":"IDR=X",
"USD/THB":"THB=X",
"USD/MYR":"MYR=X",
"USD/ZAR":"ZAR=X",
"USD/RUB":"RUB=X"
}

curr_list = list()
for key in curr_pairs:
    curr_list.append(key)
    

In [66]:
# yfinance:
curr = yf.Ticker("EURUSD=X")
# This will be a configurable call in custom function
df = curr.history(period="7d", interval="1m")
df = df[["Open","High","Low","Close"]]
df.index.rename('Time', inplace=True)
df.reset_index(inplace=True)
df['Time'] = df['Time'].dt.tz_localize(None)
df.set_index('Time', inplace=True)
# print("df.shape", df.shape)
# print("\nhead")
# print(df.head(10))
# print("\ntail")
# print(df.tail(10))
# print("\ndf start Time: {}".format(str(min(df.index))))
# print("\ndf end Time: {}".format(str(max(df.index))))

In [72]:
start_window = 1
end_window = 300
results_dict = dict()
for x in range(start_window,end_window):
    env = gym.make('forex-v0',
             df = df,
             window_size = x,
             frame_bound = (x, df.shape[0]),
             unit_side = 'right'
            )

#     print("custom env information:")
#     print("> shape:", custom_env.shape)
#     print("> df.shape:", custom_env.df.shape)
#     print("> prices.shape:", custom_env.prices.shape)
#     print("> signal_features.shape:", custom_env.signal_features.shape)
#     print("> max_possible_profit:", custom_env.max_possible_profit())
    results_dict[x] = env.max_possible_profit()

In [73]:
results_dict

{1: 0.9568200864454509,
 2: 0.9567330078063486,
 3: 0.9567330078063486,
 4: 0.9567330078063486,
 5: 0.9567330078063486,
 6: 0.9568200864454509,
 7: 0.9568200864454509,
 8: 0.9568200864454509,
 9: 0.9568200864454509,
 10: 0.9568200864454509,
 11: 0.9568200864454509,
 12: 0.9568200864454509,
 13: 0.9568200864454509,
 14: 0.956994275428966,
 15: 0.956994275428966,
 16: 0.956994275428966,
 17: 0.9571684961235619,
 18: 0.9571684961235619,
 19: 0.9569072126396351,
 20: 0.9570813857762654,
 21: 0.9570813857762654,
 22: 0.9570813857762654,
 23: 0.9570813857762654,
 24: 0.9570813857762654,
 25: 0.9569942912830602,
 26: 0.9570813857762654,
 27: 0.9570813857762654,
 28: 0.9570813857762654,
 29: 0.9570813857762654,
 30: 0.9570813857762654,
 31: 0.9570813857762654,
 32: 0.9570813857762654,
 33: 0.9572556223292912,
 34: 0.9572556223292912,
 35: 0.9574299064662645,
 36: 0.9574299064662645,
 37: 0.9572556540490367,
 38: 0.9572556540490367,
 39: 0.9572556540490367,
 40: 0.9572556540490367,
 41: 0.95725

In [52]:
def test_returns(num_tests=10, window_min=1, window_max=1000):
    max_profits_dict = dict()
    profits_dict = dict()
    for x in range(0,num_tests):
        for y in range(window_min, window_max):
            env = gym.make('forex-v0',
                     df = df,
                     window_size = y,
                     frame_bound = (y, df.shape[0]),
                     unit_side = 'left'
                    )
            observation = env.reset()
            i = 0
            while True:
                action = env.action_space.sample()
                observation, reward, done, info = env.step(action)
        #         if i % window_size == 0:
        #             print("info {}:".format(i), info)
                i += 1
                if done:
        #             print("final info:", info)
                    break
                profits_dict[y] = info['total_profit']
        max_profits_dict[max(profits_dict, key=profits_dict.get)] = profits_dict[max(profits_dict, key=profits_dict.get)]
        print("test:{}: best window val: {}, profit: {}".format(x, max(profits_dict, key=profits_dict.get), profits_dict[max(profits_dict, key=profits_dict.get)]))
    return max_profits_dict

In [54]:
test = test_returns(1,1,300)
assert test is not None
assert type(test) is dict

test 0: best window val: 180, profit: 0.7387272500960014
test 1: best window val: 280, profit: 0.7290572369607229
test 2: best window val: 90, profit: 0.7250390030851108
test 3: best window val: 292, profit: 0.7683902150756573
test 4: best window val: 33, profit: 0.7584037364771785


In [53]:
test

{98: 0.7477081332529466,
 296: 0.7495069846720265,
 5: 0.7368785518208684,
 203: 0.7435517485954636,
 152: 0.7215509392906265}